# Welcome back Commander!

See the basics in [part 1](https://www.kaggle.com/bovard/kore-fleets-intro-part-1-the-basics/), and mining in [part 2](https://www.kaggle.com/bovard/kore-intro-part-2-mining-kore)!

In [ ]:
!pip install kaggle-environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
print(env.name, env.version)

By this time, you know how fleets move and how to mine kore. Next we'll learn how to expand your empire and make new shipyards!

To create a shipyard, you need a fleet with at least 50 (`config.convert_cost`) that has a flight plan that includes the `C` (convert) command.

> When converting into a shipyard, any kore on the cell underneath the new shipyard will be destroyed. Any kore carried by the fleet converting itself will be deposited instantly and the fleet will lose 50 ships. Any remaining ships are added to the shipyard’s ship count.

Let's start with our pilot bot from [part 1](https://www.kaggle.com/bovard/kore-fleets-intro-part-1-the-basics?scriptVersionId=90399125&cellId=15)

In [ ]:
%%writefile pilot.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if shipyard.ship_count >= 50:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(50, flight_plan)
            shipyard.next_action = action
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/pilot.py"])
env.render(mode="ipython", width=1000, height=800)

This bot will allow us to gather enough resources to found a new shipyard without running out of core! Let's gate the creation of a new shipyard to when we have enough kore to build another fleet: 500.

## Expander

In [ ]:
%%writefile expander.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    convert_cost = board.configuration.convert_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        action = None
        if kore_left >= 500 and shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(10, 15))
            flight_plan = flight_plan[:6] + "C"
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/expander.py"])
env.render(mode="ipython", width=1000, height=800)

## Fin!

Now you know how to create a new shipyard! Happy coding!

Next up is [combat](https://www.kaggle.com/bovard/kore-intro-part-4-combat)!